Creates data for figure 4 and saves it to figure4.jld and respective csv files

# Figure 4a - Original model reduction results

* Panel 1 - Orig model in orig space
* Panel 2 - Red model in orig space (reproduces panel 1)
* Panel 3 - Orig model in log space
* Panel 4 - Red model in log space (DOES NOT reproduce panel 3)

In [59]:
# Initialise model and load results from figure 3
include("../BK_functions/bk_setup_script.jl"); 

using JLD
outp = load("figure3.jld", "outp");

In [68]:
#=
# Look at 3->2 param reduced model
phi_redto3 = exp( [    -55.91,       1.99,      59.45]) # With -1 direction (stops voltage sensitivy)
phi_redto3 = exp([      0.14,      -6.53,      -0.95]) # With +1 direction
res_red = BK_simulator(phi_redto3, hcat(x_grid...), model_id=15)
res = BK_simulator(phi0, hcat(x_grid...), model_id=10)
res = reshape(res, (8,13))
res_red = reshape(res_red, (8,13))
using NBInclude
nbinclude("MBAM_plots.ipynb")
import MBAM_plots
import PlotlyJS
PlotlyJS.plot([PlotlyJS.Plot(MBAM_plots.plot(V, res')), PlotlyJS.Plot(MBAM_plots.plot(V, res_red'))])
=#


In [60]:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    println(ca_str)
    
    # Generate grid to evaluate over
    x_grid_cur = Array(Any, (1*length(V),))
    iterind = 0;
    for j1 in product([ca], V)
        iterind += 1;
        x_grid_cur[iterind] = collect(j1)
    end
    
    # Get the function values for original model
    tmp = BK_simulator(exp(outp[1][1,1]), hcat(x_grid_cur...), model_id=10)
    writecsv("CSV/figure_4a_panel1_ca$(ca_str)um.csv", hcat(V[:], tmp[:]))
    
    # Get the function values for the reduced model
    tmp_red = BK_simulator(exp(outp[1][5,1]), hcat(x_grid_cur...), model_id=14)
    writecsv("CSV/figure_4a_panel2_ca$(ca_str)um.csv", hcat(V[:], tmp_red[:]))
    
    # Save also the log10 values
    writecsv("CSV/figure_4a_panel3_ca$(ca_str)um.csv", hcat(V[:], log10(tmp[:])))
    writecsv("CSV/figure_4a_panel4_ca$(ca_str)um.csv", hcat(V[:], log10(tmp_red[:])))
end
    

0.0
0.7
4.0
12.0
22.0
55.0
70.0
95.0


# Figure 4b - Log model reduction results

* Panel 1 - Orig model in log space (same as fig 4a panel 3)
* Panel 2 - 1st red in log space
* Panel 3 - 2nd red in log space
* Panel 4 - 3rd red in log space

Either panel 3 or 4 will already not reproduce panel 1, similarly to fig4 panel 4

In [61]:
# Similarly to figure 3, initalise the model, and do the model reduction, 
# but now with observing the outputs on a log10 scale
include("../BK_functions/bk_setup_script.jl"); 

In [62]:
#=
outp_log10 = MBAM(BK_simulator, phi0, x_grid, model_iters=[51, 52, 53, 54, 55], 
move_dir=[-1, -1, 1, 1, -1], boundary_time=10, verbose=1, reduce_func=BK_reduce)

using JLD
save("figure4.jld", "outp_log10", outp_log10)
=#

# Alternatively, load figure4.jld which stores this outp
using JLD
outp_log10 = load("figure4.jld", "outp_log10");

In [63]:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    println(ca_str)
    
    # Generate grid to evaluate over
    x_grid_cur = Array(Any, (1*length(V),))
    iterind = 0;
    for j1 in product([ca], V)
        iterind += 1;
        x_grid_cur[iterind] = collect(j1)
    end
    
    
    # Get the log function values for original model
    tmp = BK_simulator(exp(outp_log10[1][1,1]), hcat(x_grid_cur...), model_id=51)
    writecsv("CSV/figure_4b_panel1_ca$(ca_str)um.csv", hcat(V[:], tmp[:]))
    
    for model_red = 1:3
        # Get the log function values for reduced models
        tmp = BK_simulator(exp(outp_log10[1][1+model_red,1]), hcat(x_grid_cur...), model_id=(51+model_red))
        writecsv("CSV/figure_4b_panel$(model_red+1)_ca$(ca_str)um.csv", hcat(V[:], tmp[:]))
    end
end

0.0
0.7
4.0
12.0
22.0
55.0
70.0
95.0
